# Nested Parquet Serialization

This notebooks walks through the implementation of serialization support between nested-pandas and parquet

In [1]:
from nested_pandas.datasets.generation import generate_data

nf = generate_data(100,50, seed=1) #.rename({"b":"flux"}, axis=1)
nf

a         b                                             nested
0   0.417022  0.653290  [{t: 19.003522, flux: 79.385006, band: 'r'}; …...
1   0.720324  1.054116  [{t: 11.133064, flux: 49.858364, band: 'r'}; …...
..       ...       ...                                                ...
98  0.002870  1.233557  [{t: 19.656342, flux: 33.822987, band: 'g'}; …...
99  0.617145  1.898033  [{t: 3.632257, flux: 65.051631, band: 'g'}; …]...

[100 rows x 3 columns]

## Nested-Pandas Parquet Writing

The core change to writing is that nested columns now write to disk as pure arrow dtypes rather than their nested-pandas NestedDtype.

In [2]:
nf.to_parquet("nf.parquet")

This means that the resulting parquet files should be readable by other parquet readers, without any dependency on nested-pandas.

In [3]:
import pandas as pd
pd.read_parquet("nf.parquet")

,a,b,nested
0,0.417022,0.653290,"{'t': [19.003522384941594, 16.237173954410796,..."
1,0.720324,1.054116,"{'t': [11.133063763903127, 17.499232899117963,..."
...,...,...,...
98,0.002870,1.233557,"{'t': [19.6563422746089, 10.222829565942565, 1..."
99,0.617145,1.898033,"{'t': [3.6322570266152754, 10.819036099243773,..."


## Nested-Pandas Parquet Reading

Conversely, nested-pandas will now try to cast any struct column it reads as a nested column:

In [4]:
import nested_pandas as npd
npd.read_parquet("nf.parquet")

a         b                                             nested
0   0.417022   0.65329  [{t: 19.003522, flux: 79.385006, band: 'r'}; …...
1   0.720324  1.054116  [{t: 11.133064, flux: 49.858364, band: 'r'}; …...
..       ...       ...                                                ...
98   0.00287  1.233557  [{t: 19.656342, flux: 33.822987, band: 'g'}; …...
99  0.617145  1.898033  [{t: 3.632257, flux: 65.051631, band: 'g'}; …]...

[100 rows x 3 columns]

There are structs that exist that will not be castable:

In [5]:
import pyarrow as pa
import pyarrow.parquet as pq
# Dummy data for the struct array
field1 = pa.array([1, 2, 3])  # First field (e.g., integers)
field2 = pa.array(["a", "b", "c"])  # Second field (e.g., strings)
field3 = pa.array([[True], [False], [True, False]])  # Third field (e.g., booleans), mismatched inner length for nesting!

# Field names for the struct
field_names = ["field1", "field2", "field3"]

# Create the StructArray
struct_array = pa.StructArray.from_arrays(
    [field1, field2, field3],  # List of child arrays
    field_names  # List of field names
)

bad_tab = pa.Table.from_arrays([struct_array], names=["not_nestable"])
pq.write_table(bad_tab, "./not_nestable.parquet")

In [6]:
npd.read_parquet("not_nestable.parquet")

ValueError: Column 'not_nestable' is a Struct, but an attempt to cast it to a NestedDType failed. This is likely due to the struct not meeting the requirements for a nested column (all fields should be equal length). To proceed, you may add the column to the `reject_nesting` argument of the read_parquet function to skip the cast attempt.

In these cases, you can use the `reject_nesting` kwarg to avoid the cast:

In [7]:
npd.read_parquet("not_nestable.parquet", reject_nesting=["not_nestable"])

,not_nestable
0,"{'field1': 1, 'field2': 'a', 'field3': array([ True])}"
1,"{'field1': 2, 'field2': 'b', 'field3': array([False])}"
2,"{'field1': 3, 'field2': 'c', 'field3': array([ True, False])}"


### Partial Loading

This implementation allows for partial loading of nested columns, based on the inherent ability of pyarrow. These partial columns are loaded as part of their parent nested structure.

In [10]:
npd.read_parquet("nf.parquet", columns = ["a", "nested.flux", "nested.t"])

a                                          nested
0   0.417022  [{flux: 79.385006, t: 19.003522}; …] (50 rows)
1   0.720324  [{flux: 49.858364, t: 11.133064}; …] (50 rows)
..       ...                                             ...
98   0.00287  [{flux: 33.822987, t: 19.656342}; …] (50 rows)
99  0.617145   [{flux: 65.051631, t: 3.632257}; …] (50 rows)

[100 rows x 2 columns]

In default pyarrow (and pyarrow backed pandas), this partial loading yields top-level columns. We can emulate this by rejecting the cast:

In [9]:
npd.read_parquet("nf.parquet", columns = ["a", "nested.flux"], reject_nesting="nested")

,a,flux
0,0.417022,[79.38500585 40.40960185 97.43260633 45.73886761 53.50943617 23.26886092 12.17295054 68.81278584 65.75679317 85.96298951 1.05245356 46.06799024 69.68718793 49.38814564 76.0313713 48.55138816 72.62307379 3.89834165 17.70310546 3.18972627 82.83803856 61.76555119 70.84870338 83.70592237 51.53436626 59.1631167 30.17664139 83.58960386 36.46636848 80.43415962 72.78153978 42.48997207 34.17744787 12.4699611 30.94226416 58.08173692 49.42050979 3.4634874 24.08941287 75.93284933 91.40568673 92.62445641 12.27422566 88.79597855 74.3747174 24.78702726 95.29664038 43.72681526 76.54969605 70.79274707]
1,0.720324,[49.8583643 64.01644972 75.16724176 43.35527126 67.65410399 16.16016491 84.29710736 1.94226314 90.9170421 69.27604651 58.90978945 2.46073205 40.39730452 90.01960403 78.81383365 78.62791636 72.09270818 21.54657905 40.17564302 95.28097952 67.19999358 63.9878093 49.88484607 24.27712838 44.68538399 25.05399028 45.01985286 34.45927726 67.27495296 63.11817287 49.20431144 9.89996645 84.42799557 85.5707907 36.63582954 9.43460574 54.15293215 21.88043926 54.52347667 97.32461632 78.63451487 23.41631828 92.1632951 28.81286712 32.88645657 11.18413807 11.23269736 86.32012457 30.81727717 94.80172178]
...,...,...
98,0.00287,[33.82298681 45.98679873 21.00738531 94.05764149 24.40165353 40.54476915 45.1566674 95.08279893 91.24921775 78.56712631 47.40663572 94.7036813 2.51534163 43.25392717 36.43106573 40.90777124 29.57435395 73.06413509 80.15056135 46.91105367 36.44457691 46.484612 23.4800188 63.35788899 42.61749552 29.18082137 96.1368863 68.42307262 55.16786185 81.50331153 60.72527151 7.00456274 56.72054953 91.87235467 46.60440737 81.06128913 43.48957621 92.02304364 10.44156 57.74219543 7.65789078 27.59673184 78.72460006 24.70785943 55.58672622 23.92855217 50.83709152 10.80436563 76.33602439 47.30929466]
99,0.617145,[65.05163127 55.797723 22.68011162 79.72450194 2.04865724 84.96945345 1.85339447 56.99176424 95.76095883 54.41586104 23.2388157 40.92633156 61.28016377 52.4095811 77.40354145 84.30068052 85.99617396 77.78326997 25.31148334 94.73422535 50.21683769 95.54864986 59.65834477 10.11961778 78.29488655 49.85757767 34.20574059 82.13913121 80.79054192 26.28420186 72.95936716 37.63223351 11.09682742 8.09163877 54.31391122 31.64207823 61.32994478 53.31008527 21.76274468 48.55807291 83.36720214 89.5584923 39.89431166 25.43298582 91.80691983 7.30797255 8.47333803 76.27937776 86.4040213 7.67260741]


Notably, we lose the top-level column name, which will cause issues between base columns and sub columns that use the same name:

In [11]:
nf = generate_data(100,50, seed=1).rename({"b":"flux"}, axis=1)
nf.to_parquet("duplicate.parquet")

In [14]:
npd.read_parquet("duplicate.parquet", columns=["flux", "nested.flux"], reject_nesting="nested")

ArrowNotImplementedError: Unsupported cast from double to list using function cast_list

But this is not an issue when allowing the NestedDtype cast:

In [15]:
npd.read_parquet("duplicate.parquet", columns=["flux", "nested.flux"])

flux                            nested
0    0.65329  [{flux: 79.385006}; …] (50 rows)
1   1.054116  [{flux: 49.858364}; …] (50 rows)
..       ...                               ...
98  1.233557  [{flux: 33.822987}; …] (50 rows)
99  1.898033  [{flux: 65.051631}; …] (50 rows)

[100 rows x 2 columns]

## Performance

This implementation shifts nested-pandas from using pandas parquet reader to instead use pyarrow's parquet reader. I had some concerns about this, as it means a conversion from a pyarrow table to a pandas dataframe is needed both ways, and this operations is [only zero-copy under certain conditions](https://arrow.apache.org/docs/python/pandas.html#zero-copy-series-conversions). I implemented a few checks to try to limit any potential speed/memory regressions:

In [2]:
from nested_pandas.datasets.generation import generate_data
nf = generate_data(1000,100, seed=1) #.rename({"b":"flux"}, axis=1)
nf.to_parquet("serial_test.parquet")

### Memory
Best to restart the kernel before running either of these cells

In [1]:
from memory_profiler import memory_usage
import pandas as pd

def myfunc():
  return pd.read_parquet("serial_test.parquet", columns=["a", "nested"], engine="pyarrow", dtype_backend="pyarrow")

    
mem = max(memory_usage(proc=myfunc))

print("Maximum memory used: {} MiB".format(mem))

Maximum memory used: 169.359375 MiB


In [1]:
from memory_profiler import memory_usage
import nested_pandas as npd

def myfunc():
  return npd.read_parquet("serial_test.parquet", columns=["a", "nested"])

mem = max(memory_usage(proc=myfunc))

print("Maximum memory used: {} MiB".format(mem))

Maximum memory used: 170.609375 MiB


### Time

In [3]:
import pandas as pd

In [8]:
%%timeit
pd.read_parquet("serial_test.parquet", columns=["a", "nested"], engine="pyarrow", dtype_backend="pyarrow")

2.46 ms ± 5.19 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
npd.read_parquet("serial_test.parquet", columns=["a", "nested"])

2.81 ms ± 9.22 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Remote Filesystems: using UPath

In [11]:
npd.read_parquet("https://epyc.astro.washington.edu/~lincc-frameworks/hipscat_surveys/erosita/erosita_dr1_erass1/Norder=0/Dir=0/Npix=2.parquet")

,IAUNAME,DETUID,SKYTILE,ID_SRC,UID,UID_Hard,ID_CLUSTER,RA,DEC,RA_RAW,DEC_RAW,RA_LOWERR,RA_UPERR,DEC_LOWERR,DEC_UPERR,POS_ERR,RADEC_ERR,LII,BII,ELON,ELAT,MJD,MJD_MIN,MJD_MAX,EXT,EXT_ERR,EXT_LOWERR,EXT_UPERR,EXT_LIKE,DET_LIKE_0,ML_CTS_1,ML_CTS_ERR_1,ML_CTS_LOWERR_1,ML_CTS_UPERR_1,ML_RATE_1,ML_RATE_ERR_1,ML_RATE_LOWERR_1,ML_RATE_UPERR_1,ML_FLUX_1,ML_FLUX_ERR_1,ML_FLUX_LOWERR_1,ML_FLUX_UPERR_1,ML_BKG_1,ML_EXP_1,ML_EEF_1,APE_CTS_1,APE_BKG_1,APE_EXP_1,APE_RADIUS_1,APE_POIS_1,DET_LIKE_P1,ML_CTS_P1,ML_CTS_ERR_P1,ML_CTS_LOWERR_P1,ML_CTS_UPERR_P1,ML_RATE_P1,ML_RATE_ERR_P1,ML_RATE_LOWERR_P1,ML_RATE_UPERR_P1,ML_FLUX_P1,ML_FLUX_ERR_P1,ML_FLUX_LOWERR_P1,ML_FLUX_UPERR_P1,ML_BKG_P1,ML_EXP_P1,ML_EEF_P1,APE_CTS_P1,APE_BKG_P1,APE_EXP_P1,APE_RADIUS_P1,APE_POIS_P1,DET_LIKE_P2,ML_CTS_P2,ML_CTS_ERR_P2,ML_CTS_LOWERR_P2,ML_CTS_UPERR_P2,ML_RATE_P2,ML_RATE_ERR_P2,ML_RATE_LOWERR_P2,ML_RATE_UPERR_P2,ML_FLUX_P2,ML_FLUX_ERR_P2,ML_FLUX_LOWERR_P2,ML_FLUX_UPERR_P2,ML_BKG_P2,ML_EXP_P2,ML_EEF_P2,APE_CTS_P2,APE_BKG_P2,APE_EXP_P2,APE_RADIUS_P2,APE_POIS_P2,DET_LIKE_P3,ML_CTS_P3,ML_CTS_ERR_P3,ML_CTS_LOWERR_P3,ML_CTS_UPERR_P3,ML_RATE_P3,ML_RATE_ERR_P3,ML_RATE_LOWERR_P3,ML_RATE_UPERR_P3,ML_FLUX_P3,ML_FLUX_ERR_P3,ML_FLUX_LOWERR_P3,ML_FLUX_UPERR_P3,ML_BKG_P3,ML_EXP_P3,ML_EEF_P3,APE_CTS_P3,APE_BKG_P3,APE_EXP_P3,APE_RADIUS_P3,APE_POIS_P3,DET_LIKE_P4,ML_CTS_P4,ML_CTS_ERR_P4,ML_CTS_LOWERR_P4,ML_CTS_UPERR_P4,ML_RATE_P4,ML_RATE_ERR_P4,ML_RATE_LOWERR_P4,ML_RATE_UPERR_P4,ML_FLUX_P4,ML_FLUX_ERR_P4,ML_FLUX_LOWERR_P4,ML_FLUX_UPERR_P4,ML_BKG_P4,ML_EXP_P4,ML_EEF_P4,APE_CTS_P4,APE_BKG_P4,APE_EXP_P4,APE_RADIUS_P4,APE_POIS_P4,DET_LIKE_P5,ML_CTS_P5,ML_CTS_ERR_P5,ML_CTS_LOWERR_P5,ML_CTS_UPERR_P5,ML_RATE_P5,ML_RATE_ERR_P5,ML_RATE_LOWERR_P5,ML_RATE_UPERR_P5,ML_FLUX_P5,ML_FLUX_ERR_P5,ML_FLUX_LOWERR_P5,ML_FLUX_UPERR_P5,ML_BKG_P5,ML_EXP_P5,ML_EEF_P5,APE_CTS_P5,APE_BKG_P5,APE_EXP_P5,APE_RADIUS_P5,APE_POIS_P5,DET_LIKE_P6,ML_CTS_P6,ML_CTS_ERR_P6,ML_CTS_LOWERR_P6,ML_CTS_UPERR_P6,ML_RATE_P6,ML_RATE_ERR_P6,ML_RATE_LOWERR_P6,ML_RATE_UPERR_P6,ML_FLUX_P6,ML_FLUX_ERR_P6,ML_FLUX_LOWERR_P6,ML_FLUX_UPERR_P6,ML_BKG_P6,ML_EXP_P6,ML_EEF_P6,APE_CTS_P6,APE_BKG_P6,APE_EXP_P6,APE_RADIUS_P6,APE_POIS_P6,DET_LIKE_P7,ML_CTS_P7,ML_CTS_ERR_P7,ML_CTS_LOWERR_P7,ML_CTS_UPERR_P7,ML_RATE_P7,ML_RATE_ERR_P7,ML_RATE_LOWERR_P7,ML_RATE_UPERR_P7,ML_FLUX_P7,ML_FLUX_ERR_P7,ML_FLUX_LOWERR_P7,ML_FLUX_UPERR_P7,ML_BKG_P7,ML_EXP_P7,ML_EEF_P7,APE_CTS_P7,APE_BKG_P7,APE_EXP_P7,APE_RADIUS_P7,APE_POIS_P7,DET_LIKE_P8,ML_CTS_P8,ML_CTS_ERR_P8,ML_CTS_LOWERR_P8,ML_CTS_UPERR_P8,ML_RATE_P8,ML_RATE_ERR_P8,ML_RATE_LOWERR_P8,ML_RATE_UPERR_P8,ML_FLUX_P8,ML_FLUX_ERR_P8,ML_FLUX_LOWERR_P8,ML_FLUX_UPERR_P8,ML_BKG_P8,ML_EXP_P8,ML_EEF_P8,APE_CTS_P8,APE_BKG_P8,APE_EXP_P8,APE_RADIUS_P8,APE_POIS_P8,DET_LIKE_P9,ML_CTS_P9,ML_CTS_ERR_P9,ML_CTS_LOWERR_P9,ML_CTS_UPERR_P9,ML_RATE_P9,ML_RATE_ERR_P9,ML_RATE_LOWERR_P9,ML_RATE_UPERR_P9,ML_FLUX_P9,ML_FLUX_ERR_P9,ML_FLUX_LOWERR_P9,ML_FLUX_UPERR_P9,ML_BKG_P9,ML_EXP_P9,ML_EEF_P9,APE_CTS_P9,APE_BKG_P9,APE_EXP_P9,APE_RADIUS_P9,APE_POIS_P9,APE_CTS_S,APE_BKG_S,APE_EXP_S,APE_POIS_S,FLAG_SP_SNR,FLAG_SP_BPS,FLAG_SP_SCL,FLAG_SP_LGA,FLAG_SP_GC_CONS,FLAG_NO_RADEC_ERR,FLAG_NO_EXT_ERR,FLAG_NO_CTS_ERR,FLAG_OPT,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2305850970984153088,b'1eRASS J150021.3+001057',b'eb01_227090_020_ML00154_002_c010',227090,154,122709000154,0,144,225.089055,0.18273,225.089864,0.182042,3.024563,3.072646,3.63202,3.631488,3.590569,4.311378,357.257112,48.963814,222.569426,16.53664,58881.492188,58880.824219,58882.324219,0.0,0.0,0.0,0.0,0.0,16.201927,9.78603,3.505255,3.101283,3.909226,0.060328,0.021609,0.019118,0.024099,0.0,0.0,0.0,0.0,1.363216,162.213745,0.883602,8,1.102543,162.228546,7.63742,0.00002,3.596417,1.986781,1.528363,1.146522,1.910204,0.011986,0.00922,0.006917,0.011524,0.0,0.0